# 2. Tuning Part 2

### for SVM and Log Regression

In [1]:
import pandas as pd
import re
import nltk
import copy
from sklearn.model_selection import train_test_split
import copy
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [2]:
# df = pd.read_csv('Phase1.csv')
df = pd.read_csv('Phase1_1.csv')
df1 = pd.read_csv('Phase1_1test.csv')

In [3]:
text_list = list()
for tweet in df['text']:
    
    tmp = list()
    
    # get rid of non-alphanumerical chars, effectively splitting them
    # except for at ' - # and %
    tweet = re.sub(r'[^\x00-\x7F#%\'-]', ' ', tweet)

    # tweet = re.sub(r'[^\w%\'-]', ' ', tweet)
                
#     # get rid of non-english characters
#     tweet = re.sub(r'[^\x00-\x7F ]+', ' ', tweet) 
    
#     # ??? 是不是有问题
    
    # split the words based on space
    words = tweet.split()
    
    for word in words:
        
        # get rid of ' and % and - that are outside of words
        word = word.strip('\'%-')
        
        # get rid of ' in all words
        # CURRENTLY KEEPS - in words
        word = re.sub(r'[^\w#-]', '', word)
        
        # word = re.sub(r'[^\w-]', '', word)
        
        if word not in ['', '#']:
            tmp.append(word)
    text_list.append(tmp)

In [4]:
df['text'] = text_list
df.to_csv('phase2.csv', index=False)

In [5]:
### THIS IS FOR THE PROCESSING OF TESTING DATA (remove punctuation) IF NEEDED DEPENDING ON OUR DECISION OF 
# USING BAILAN OR PREPROCESS
text_list = list()
for tweet in df1['text']:
    
    tmp = list()
    
    # get rid of non-alphanumerical chars, effectively splitting them
    # except for at ' - # and %
    tweet = re.sub(r'[^\x00-\x7F#%\'-]', ' ', tweet)

    # tweet = re.sub(r'[^\w%\'-]', ' ', tweet)
                
#     # get rid of non-english characters
#     tweet = re.sub(r'[^\x00-\x7F ]+', ' ', tweet) 
    
#     # ??? 是不是有问题
    
    # split the words based on space
    words = tweet.split()
    
    for word in words:
        
        # get rid of ' and % and - that are outside of words
        word = word.strip('\'%-')
        
        # get rid of ' in all words
        # CURRENTLY KEEPS - in words
        word = re.sub(r'[^\w#-]', '', word)
        
        # word = re.sub(r'[^\w-]', '', word)
        
        if word not in ['', '#']:
            tmp.append(word)
    text_list.append(tmp)
df1['text'] = text_list
display(df1)

,id,text
0,802217876644052000,"[putin, abducted, ukrainian, citizens, in, occ..."
1,802425296955682000,"[if, putin, wanted, to, intervene, all, he, wo..."
2,805664502515662000,"[elsewhere, in, progland]"
3,639928670103015000,"[any, females, copping, the, gucci, foamposite..."
4,673824182287904000,"[in, other, news, had, a, great, saturday, on,..."
...,...,...
6094,662664783083122000,"[i, have, tickets, to, see, the, vamps, on, ap..."
6095,635582180316479000,"[you, may, have, seen, this, but, john, cena, ..."
6096,805537394262994000,"[this, is, what, democracy, should, look, like..."
6097,640586501571637000,"[damn, shawn, was, right, i, may, have, to, co..."


## Stemming

In [6]:
from nltk.stem import PorterStemmer
# stemmeddf = df.copy(deep=True)
ps = PorterStemmer()

def stem_text(text):
    return [ps.stem(w) for w in text]

# stemmeddf['text'] = stemmeddf.text.apply(stem_text)

In [7]:
# stemmeddf.to_csv('stemmedPhase2.csv', index = False)

## Stopwords

In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

#rmstopworddf = df.copy(deep=True)
def remove_stopwords(text):
    return [w for w in text if not w in stop_words]

#rmstopworddf['text'] = rmstopworddf.text.apply(remove_stopwords)

In [9]:
#rmstopworddf.to_csv('rmStopwordsPhase2.csv', index = False)

## TFIDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf_vectorizer = TfidfVectorizer()

#Build the feature set (vocabulary) and vectorise the Tarin dataset using TFIDF
# X_train_tfidf = tfidf_vectorizer.fit_transform(df_for_vectorize['text'])


# print("Train feature space size (using TFIDF):",X_train_tfidf.shape)

In [11]:
# print(X_train_tfidf[0])

### dataset


In [12]:
# BoW_vectorizer = CountVectorizer(lowercase=False, ngram_range=(1,1))

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]
def join_string(text):
    return ' '.join(text)
def remove_hashtag(text):
    return [w.strip("#") for w in text]

dfcopy = df.copy(deep = True)
# dfcopy['text'] = dfcopy['text'].apply(remove_stopwords)
dfcopy['text'] = dfcopy['text'].apply(remove_hashtag)
dfcopy['text'] = dfcopy['text'].apply(stem_text)
dfcopy['text'] = dfcopy['text'].apply(join_string)
print(dfcopy['text'][0])
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1))
X_train_processed = tfidf_vectorizer.fit_transform(dfcopy['text'])
Y_train_processed = dfcopy['sentiment']
print(X_train_processed.shape)

#UNPROCESSED only tfidf
dfcopy1 = pd.read_csv("Train.csv")
#tfidf_vectorizer = TfidfVectorizer()
X_train_UN = tfidf_vectorizer.fit_transform(dfcopy1['text'])
Y_train_UN = dfcopy1['sentiment']

X_word_2vec = pd.read_csv("W2V_Train.csv")
# # Expand contraction new function in phase 1
# # Xtra processed
# dfcopy2 = df1.copy(deep = True)
# dfcopy2['text'] = dfcopy2['text'].apply(stem_text)
# # dfcopy['text'] = dfcopy['text'].apply(remove_stopwords)
# dfcopy2['text'] = dfcopy2['text'].apply(join_string)
# tfidf_vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1))
# X_train_Xprocessed = tfidf_vectorizer.fit_transform(dfcopy2['text'])
# Y_train_Xprocessed = dfcopy2['sentiment']

doctor hit campaign trail as race to medic council elect heat up homeopathi
(21801, 21734)


In [13]:
print(X_word_2vec.shape)
print(X_train_processed.shape)

(21800, 99)
(21801, 21734)


# MODEL

## Logistice Reg tunning parameter

In [1]:
import copy
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
accu_score = list()
lst = []
switch = True
i = 0
X = copy.deepcopy(X_word_2vec)
y = copy.deepcopy(Y_train_processed)
while switch:
    X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, train_size = 0.7, random_state = 30027)
    X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, train_size = 0.5, random_state = 30027)
    X2 = SelectKBest(chi2, k=int(X.shape[1]*0.25)+i)
    X2.fit(X_train, y_train)
    X_train = X2.transform(X_train)
    X_val = X2.transform(X_val)
    print('a')
    parameter_space = {
        'penalty': ['l2', 'none', 'l1', 'elasticnet'],
        'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2],
        'solver': ['saga'],
        'multi_class': ['auto', 'ovr', 'multinomial']
    }
    lgr = LogisticRegression(max_iter = 1000)
    clf = GridSearchCV(lgr, parameter_space, n_jobs=-1, cv=2)
    
    clf.fit(X_train, y_train)
    print('Best parameters found:\n', clf.best_params_)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, params in zip(means, clf.cv_results_['params']):
        lst.append((((X.shape[1]*0.25)+i, mean), params))
    i += 1000
    if int(X.shape[1]*0.25)+i >= X.shape[1]:
        switch = False

In [ ]:
print('Best parameters found:\n', clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))


## SVM tunning parameter

In [ ]:
from sklearn import svm
accu_score = list()
X = copy.deepcopy(X_train_processed)
y = copy.deepcopy(Y_train_processed)

X_train, X_valtest, y_train, y_valtest = train_test_split(X, y, train_size = 0.7, random_state = 30027)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, train_size = 0.5, random_state = 30027)
svm = svm.SVC()
parameter_space = {'C': [0.1,1, 10, 100], 
                    'gamma': [1,0.1,0.01,0.001],
                    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                    'degree':[1,2,3,4,5,6,7,8,9,10],
                    'decision_function_shape':['ovo', 'ovr']
                       
                   }

clf = GridSearchCV(svm, parameter_space, n_jobs=-1, cv=2)
print('a')
clf.fit(X_train, y_train)
print('b')

# {'C': 10, 'decision_function_shape': 'ovo', 'degree': 1, 'gamma': 0.1, 'kernel': 'poly'}
        

print('Best parameters found:\n', clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))